In [ ]:
# Create a Python connection with SQL database and retrieve the results of the last two queries in sql file as dataframes.
# Hint: You can store the results from the two queries in two separate dataframes.

# Write a function that checks if customer borrowed more or less films in the month of June as compared to May.
# Hint: For this part, you can create a join between the two dataframes created before, using the merge function available for pandas dataframes. 

In [1]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [10]:
import getpass
password = getpass.getpass()

········


In [11]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [22]:
data = pd.read_sql_query('select * from sakila.rental', engine)
data

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
16039,16045,2005-08-23 22:25:26,772,14,2005-08-25 23:54:26,1,2006-02-15 21:30:53
16040,16046,2005-08-23 22:26:47,4364,74,2005-08-27 18:02:47,2,2006-02-15 21:30:53
16041,16047,2005-08-23 22:42:48,2088,114,2005-08-25 02:48:48,2,2006-02-15 21:30:53
16042,16048,2005-08-23 22:43:07,2019,103,2005-08-31 21:33:07,1,2006-02-15 21:30:53


In [20]:
# Check the number of rentals for each customer for May

# Create temporary table
create_table_query = '''
CREATE TEMPORARY TABLE sakila.rentals_may AS
SELECT 
    customer_id, 
    COUNT(rental_id) AS rental_count
FROM 
    sakila.rental
WHERE 
    MONTH(rental_date) = 5
GROUP BY 
    customer_id
ORDER BY 
    rental_count DESC
'''

engine = create_engine(connection_string)
with engine.connect() as connection:
    connection.execute(create_table_query)

# Query the temporary table to get the data
select_query = 'SELECT * FROM sakila.rentals_may'

rentals_may = pd.read_sql_query(select_query, engine)
rentals_may

,customer_id,rental_count
0,197,8
1,109,7
2,506,7
3,19,6
4,53,6
...,...,...
515,580,1
516,582,1
517,590,1
518,595,1


In [25]:
# Check the number of rentals for each customer for June

# Create temporary table
create_table_query2 = '''
CREATE TEMPORARY TABLE sakila.rentals_june AS
SELECT 
    customer_id, 
    COUNT(rental_id) AS rental_count
FROM 
    sakila.rental
WHERE 
    MONTH(rental_date) = 6
GROUP BY 
    customer_id
ORDER BY 
    rental_count DESC
'''

engine = create_engine(connection_string)
with engine.connect() as connection:
    connection.execute(create_table_query2)

# Query the temporary table to get the data 
select_query2 = 'SELECT * FROM sakila.rentals_june'

# Corrected variable name
rentals_june = pd.read_sql_query(select_query2, engine)
rentals_june

,customer_id,rental_count
0,31,11
1,454,10
2,213,9
3,267,9
4,295,9
...,...,...
585,549,1
586,555,1
587,564,1
588,580,1


In [36]:
joined_rentals = rentals_may.merge(rentals_june, on='customer_id', suffixes=('_may', '_june'), how='outer')
joined_rentals

,customer_id,rental_count_may,rental_count_june
0,197,8.0,8.0
1,109,7.0,5.0
2,506,7.0,5.0
3,19,6.0,3.0
4,53,6.0,5.0
...,...,...,...
593,335,NaN,1.0
594,370,NaN,1.0
595,487,NaN,1.0
596,555,NaN,1.0


In [39]:
# Function to compare rental counts for May and June
def compare_rentals(df):
    # Create a new column to indicate the comparison result
    df['rentals_difference'] = df.apply(
        lambda row: 'borrowed_more_in_june' if row['rental_count_june'] > row['rental_count_may'] else (
            'borrowed_less_in_june' if row['rental_count_june'] < row['rental_count_may'] else 'borrowed_same_in_june'), axis=1)
    return df

rentals_diff = compare_rentals(joined_rentals)
rentals_diff = rentals_diff.dropna()
rentals_diff

,customer_id,rental_count_may,rental_count_june,rentals_difference
0,197,8.0,8.0,borrowed_same_in_june
1,109,7.0,5.0,borrowed_less_in_june
2,506,7.0,5.0,borrowed_less_in_june
3,19,6.0,3.0,borrowed_less_in_june
4,53,6.0,5.0,borrowed_less_in_june
...,...,...,...,...
515,580,1.0,1.0,borrowed_same_in_june
516,582,1.0,3.0,borrowed_more_in_june
517,590,1.0,5.0,borrowed_more_in_june
518,595,1.0,2.0,borrowed_more_in_june
